In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix, make_scorer
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth

In [6]:
nba_2017_2018 = pd.read_csv('archive-2/NBA Players Stats 201718.csv')
nba_2017_2018 = nba_2017_2018.drop(columns=['Rk','Tm', 'Pos'], axis=1)
nba_2017_2018['Player'] = nba_2017_2018['Player'].str.split('\\').str[0]
nba_2017_2018.fillna(0, inplace=True)
nba_2017_2018 = nba_2017_2018[nba_2017_2018['G'] >= 20]
cols_to_keep = ['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
                'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Group the data by player and calculate the weighted average and sum of the other columns
nba_2017_2018 = nba_2017_2018.groupby(['Player', 'Age'])[cols_to_keep].agg({'G': 'sum', 'GS': 'sum', 
                                                                                     'MP': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FG': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FGA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FG%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3P': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3PA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '3P%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2P': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2PA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     '2P%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'eFG%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FT': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FTA': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'FT%': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'ORB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'DRB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'TRB': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'AST': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']), 
                                                                                     'STL': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'BLK': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'TOV': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'PF': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G']),
                                                                                     'PTS': lambda x: np.average(x, weights=nba_2017_2018.loc[x.index, 'G'])}).reset_index()

percentage_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
for col in percentage_columns:
    nba_2017_2018[col] = nba_2017_2018[col] * 100

nba_2017_2018 = nba_2017_2018.round(2)

all_star_list = ['Kemba Walker', 'Kyrie Irving', 'Kawhi Leonard', 'Giannis Antetokounmpo', 
                'Joel Embiid', 'Kyle Lowry', 'Victor Oladipo', 'Khris Middleton', 'Bradley Beal',
                'Ben Simmons', 'Blake Griffin', 'Nikola Vučević', "D'Angelo Russell",
                'Stephen Curry', 'James Harden', 'Kevin Durant', 'Paul George', 'LeBron James',
                'Russell Westbrook', 'Damian Lillard', 'Klay Thompson', 'Anthony Davis', 
                'LaMarcus Aldridge', 'DeMarcus Cousins', 'Nikola Jokić', 'Karl-Anthony Towns']
nba_2017_2018['AS_NY'] = nba_2017_2018['Player'].apply(lambda x: 1 if x in all_star_list else 0)

In [11]:
nba_2017_2018

,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,AS_NY
0,Aaron Brooks,33,32,1,5.9,0.9,2.2,40.6,0.3,1.0,...,0.2,0.3,0.5,0.6,0.2,0.0,0.3,0.9,2.3,0
1,Aaron Gordon,22,58,57,32.9,6.5,14.9,43.4,2.0,5.9,...,1.5,6.4,7.9,2.3,1.0,0.8,1.8,1.9,17.6,0
2,Abdel Nader,24,48,1,10.9,1.0,3.1,33.6,0.5,1.4,...,0.3,1.2,1.5,0.5,0.3,0.2,0.7,0.9,3.0,0
3,Al Horford,31,72,72,31.6,5.1,10.5,48.9,1.3,3.1,...,1.4,5.9,7.4,4.7,0.6,1.1,1.8,1.9,12.9,0
4,Al Jefferson,33,36,1,13.4,3.1,5.8,53.4,0.0,0.1,...,0.8,3.1,4.0,0.8,0.4,0.6,0.6,1.8,7.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Zach Collins,20,66,1,15.8,1.7,4.4,39.8,0.5,1.7,...,0.7,2.7,3.3,0.8,0.3,0.5,0.9,2.0,4.4,0
410,Zach LaVine,22,24,24,27.3,5.7,14.8,38.3,1.8,5.1,...,0.4,3.5,3.9,3.0,1.0,0.2,1.8,2.3,16.7,0
411,Zach Randolph,36,59,57,25.6,6.1,12.9,47.3,0.9,2.5,...,1.6,5.1,6.7,2.2,0.7,0.2,2.0,2.0,14.5,0
412,Zaza Pachulia,33,69,57,14.1,2.2,3.8,56.4,0.0,0.0,...,1.3,3.4,4.7,1.6,0.6,0.2,1.0,1.8,5.4,0


In [9]:
nba_2018_2019 = pd.read_csv('archive-2/NBA Players Stats 201819.csv')
nba_2018_2019 = nba_2018_2019.drop(columns=['Rk','Tm','Pos'], axis=1)
nba_2018_2019['Player'] = nba_2018_2019['Player'].str.split('\\').str[0]
nba_2018_2019.fillna(0, inplace=True)
nba_2018_2019 = nba_2018_2019[nba_2018_2019['G'] >= 20]
cols_to_keep = ['Player', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
                '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
                'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Group the data by player and calculate the weighted average and sum of the other columns
nba_2018_2019 = nba_2018_2019.groupby(['Player', 'Age'])[cols_to_keep].agg({'G': 'sum', 'GS': 'sum', 
                                                                                     'MP': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FG': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FGA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FG%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3P': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3PA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '3P%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2P': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2PA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     '2P%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'eFG%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FT': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FTA': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'FT%': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'ORB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'DRB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'TRB': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'AST': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']), 
                                                                                     'STL': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'BLK': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'TOV': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'PF': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G']),
                                                                                     'PTS': lambda x: np.average(x, weights=nba_2018_2019.loc[x.index, 'G'])}).reset_index()

percentage_columns = ['FG%', '3P%', '2P%', 'eFG%', 'FT%']
for col in percentage_columns:
    nba_2018_2019[col] = nba_2018_2019[col] * 100


nba_2018_2019 = nba_2018_2019.round(2)

all_star_list = ['Kemba Walker', 'Trae Young', 'Giannis Antetokounmpo', 'Pascal Siakam', 'Joel Embiid',
                'Kyle Lowry', 'Ben Simmons', 'Jimmy Butler', 'Khris Middleton', 'Bam Adebayo', 
                'Jayson Tatum', 'Domantas Sabonis', 'James Harden', 'Luka Dončić', 'LeBron James',
                'Kawhi Leonard', 'Anthony Davis', 'Chris Paul', 'Russell Westbrook', 'Damian Lillard', 
                'Donovan Mitchell', 'Brandon Ingram', 'Nikola Jokić', 'Rudy Gobert', 'Devin Booker']
nba_2018_2019['AS_NY'] = nba_2018_2019['Player'].apply(lambda x: 1 if x in all_star_list else 0)

In [10]:
nba_2018_2019

,Player,Age,G,GS,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,AS_NY
0,Aaron Gordon,23,78,78,33.8,6.0,13.4,44.9,1.6,4.4,...,1.7,5.7,7.4,3.7,0.7,0.7,2.1,2.2,16.0,0
1,Aaron Holiday,22,50,0,12.9,2.1,5.2,40.1,0.9,2.5,...,0.1,1.2,1.3,1.7,0.4,0.3,0.8,1.4,5.9,0
2,Abdel Nader,25,61,1,11.4,1.5,3.5,42.3,0.5,1.6,...,0.2,1.7,1.9,0.3,0.3,0.2,0.4,1.1,4.0,0
3,Al Horford,32,68,68,29.0,5.7,10.6,53.5,1.1,3.0,...,1.8,5.0,6.7,4.2,0.9,1.3,1.5,1.9,13.6,0
4,Al-Farouq Aminu,28,81,81,28.3,3.2,7.3,43.3,1.2,3.5,...,1.4,6.1,7.5,1.3,0.8,0.4,0.9,1.8,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Yogi Ferrell,25,71,3,15.0,2.2,5.0,43.5,0.8,2.1,...,0.2,1.4,1.5,1.9,0.5,0.1,0.6,0.9,5.9,0
419,Zach Collins,21,77,0,17.6,2.5,5.2,47.3,0.5,1.6,...,1.4,2.8,4.2,0.9,0.3,0.9,1.0,2.3,6.6,0
420,Zach LaVine,23,63,62,34.5,8.4,18.0,46.7,1.9,5.1,...,0.6,4.0,4.7,4.5,1.0,0.4,3.4,2.2,23.7,0
421,Zaza Pachulia,34,68,3,12.9,1.3,2.8,44.0,0.0,0.1,...,1.5,2.4,3.9,1.3,0.5,0.3,0.8,2.2,3.9,0


**Naive Model**

In [12]:
X_train = nba_2017_2018.drop(['Player', 'AS_NY'], axis=1) 
y_train = nba_2017_2018['AS_NY']  

X_test = nba_2018_2019.drop(['Player', 'AS_NY'], axis=1)  
y_test = nba_2018_2019['AS_NY']  

y_pred = [0] * len(y_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:")
print(confusion_mat)

Accuracy: 0.9408983451536643
Recall: 0.0
Precision: 0.0
Confusion Matrix:
[[398   0]
 [ 25   0]]


/Users/davisjohnson/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Base Logistic Regression Model**

In [15]:
logreg = LogisticRegression(max_iter=10000)
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Recall:", recall)
print("Precision:", precision)
print("Confusion Matrix:")
print(confusion_mat)

Accuracy: 0.9314420803782506
Recall: 0.56
Precision: 0.4375
Confusion Matrix:
[[380  18]
 [ 11  14]]


**Feature Importances**

In [21]:
coefficients = logreg.coef_[0]


feature_names = X_train.columns


feature_importance = sorted(zip(feature_names, coefficients), key=lambda x: abs(x[1]), reverse=True)


df_feature_importance = pd.DataFrame(feature_importance, columns=['Feature', 'Coefficient'])


df_feature_importance['Absolute Importance'] = df_feature_importance['Coefficient'].abs()
df_feature_importance = df_feature_importance.sort_values(by='Absolute Importance', ascending=False)
df_feature_importance

,Feature,Coefficient,Absolute Importance
0,PTS,0.608374,0.608374
1,AST,0.595697,0.595697
2,2P,0.574017,0.574017
3,FG,0.564848,0.564848
4,FTA,-0.538899,0.538899
5,eFG%,0.442229,0.442229
6,TRB,0.414743,0.414743
7,TOV,0.392392,0.392392
8,FG%,-0.379285,0.379285
9,FT,-0.292378,0.292378
